In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, date_add_times
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature, get_factorize_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel
from itertools import combinations

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths  = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('ProductCD_t')
               or path.count('uid')
               or path.count('D')
               or path.count('V')
               )
               and not path.count('bin_')
               and not path.count('fill_')
               and not path.count('129')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('ProductCD_t')
               or path.count('uid')
               or path.count('D')
               or path.count('V')
               )
               and not path.count('bin_')
               and not path.count('fill_')
               and not path.count('129')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
train_length = df_train.shape[0]
cols_uid = [col for col in df_train.columns if col.count('uid')]

In [3]:
def parallel_uid_agg(tmp_train, tmp_test, uid, cols_feature, list_agg):
    for feature in tqdm(cols_feature):
        for agg_type in list_agg:
            new_col_name = uid.replace('130__', '') + '_' + feature + '_' + agg_type
            temp_df = pd.concat([tmp_train[[uid, feature]], tmp_test[[uid, feature]]])
            temp_df = temp_df.groupby([uid])[feature].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})

            temp_df.index = list(temp_df[uid])
            temp_df = temp_df[new_col_name].to_dict()   

            tmp_train[new_col_name] = tmp_train[uid].map(temp_df)
            tmp_test[new_col_name]  = tmp_test[uid].map(temp_df)
            
            save_feature(tmp_train[[new_col_name]], prefix, dir_save, is_train=True, auto_type=False, list_ignore=COLUMNS_IGNORE)
            save_feature(tmp_test[[new_col_name]],  prefix, dir_save, is_train=False, auto_type=False, list_ignore=COLUMNS_IGNORE)

In [5]:
cols_C = [col for col in df_train.columns if col.startswith('C')]
cols_V = [col for col in df_train.columns if col.startswith('V')]
cols_D = [col for col in df_train.columns if col.startswith('D')]

['D15',
 'D2',
 'D6',
 'D10',
 'D4',
 'D5',
 'D14',
 'D11',
 'D3',
 'D9',
 'D1',
 'D12',
 'D7',
 'D13',
 'D8']

In [6]:
def make_uid_feature(cols_feature, list_agg):
    Parallel(len(cols_uid))([
        delayed(parallel_uid_agg)(
            df_train[[uid] + cols_feature], df_test[[uid] + cols_feature], uid, cols_feature, list_agg
        ) for uid in cols_uid])

In [17]:
prefix = '612'
dir_save = 'create'
list_agg = ['mean', 'std']

# C
# make_uid_feature(cols_C, list_agg)
# V
make_uid_feature(cols_V[40:], list_agg)
# D
# make_uid_feature(cols_D, list_agg)